In [1]:
import itertools
from treys import Card
from treys import Evaluator
from treys import Deck

In [2]:
def lget(l, index, default=None):
    try:
        return l[index]
    except IndexError:
        return default

In [3]:
def typeOfCard(card, cardsDesired):
    if not cardsDesired:
        return True
    
    return card in cardsDesired

In [13]:
%reload_ext autoreload
%autoreload 2
from strategies import *

keyWords = {'FirstFive':1, 'FirstFlush':1,'FirstPair':1,'FirstTwoPair':1,'FirstTrips':1}

def simulatePokerSolitaire(cardsDesired, keyWord = None):
    playerCards, communityCards, deck = [], [], Deck()

    if keyWord is None:
        # run with cardsDesired
        while len(playerCards) < 5:
            card = deck.draw(1)
        if typeOfCard(Card.print_pretty_card(card), Card.print_pretty_cards(cardsDesired)):
            playerCards.append(card)
        else:
            communityCards.append(card)
    elif keyWord not in keyWords:
        raise SyntaxError("Invalid keyword")
    else:
        keyWord = keyWord + '(deck)'
        playerCards, communityCards = eval(keyWord)

    numOfCardsNeeded = 5 - len(communityCards)
    if numOfCardsNeeded > 0:
        cards = deck.draw(numOfCardsNeeded)
        if numOfCardsNeeded == 1:
            cards = [cards]
        communityCards = communityCards + cards

    return playerCards, communityCards

In [7]:
def findBestCombination(totalCombinations, highestScore, evaluator):
    for combination in totalCombinations:
        if evaluator.evaluate(list(combination), []) == highestScore:
            return list(combination)
    
    return None

In [8]:
def evaluateBetterHand(playerCards, communityCards):
    evaluator = Evaluator()
    playerScore = evaluator.evaluate(playerCards, [])
    totalCombinations = [combinationCommunityCards for combinationCommunityCards in itertools.combinations(communityCards, 5)]
    highestScore = min(evaluator.evaluate(list(combination), []) for combination in totalCombinations)
    bestCommunityCombination = findBestCombination(totalCombinations, highestScore, evaluator)
    
    print(playerScore, highestScore)
    return highestScore >= playerScore, bestCommunityCombination

In [9]:
def playPoker(enterHand, keyWord):
    numericCardHand = [((CHAR_RANK_TO_INT_RANK[lget(card, 0)] << 8) + 
                        (CHAR_SUIT_TO_INT_SUIT[lget(card, 1)] << 12)) for card in enterHand]
    playerCards, communityCards = simulatePokerSolitaire(numericCardHand, keyWord)
    playerWin, bestCommunityCombination = evaluateBetterHand(playerCards, communityCards)
    
    return playerWin, Card.print_pretty_cards(playerCards), Card.print_pretty_cards(communityCards), Card.print_pretty_cards(bestCommunityCombination)

In [10]:
enterHand = ['4d', '3s', 'As', '2d', '5c']

In [18]:
playPoker([], "FirstTwoPair")


2855 179


(False,
 ' [8♠],[J♥],[Q♠],[J♦],[8♣] ',
 ' [K♠],[7♠],[2♠],[2♣],[A♣],[3♦],[A♦],[K♣],[6♠],[3♣],[K♥] ',
 ' [K♠],[A♣],[A♦],[K♣],[K♥] ')